# Lecture 1 - MP/MRP/MDP Overview
### Notes and Code for Basic Data Structures
Source for notes: [CME 241 Lecture Slides](http://web.stanford.edu/class/cme241/)

## Markov Property

All relevant information for future actions/states/outcomes is dependent only on the current action/state/outcome. That is, the string of all historical steps in a process does not matter at any point - only the relevant information for the observed period matters. Formally, this means that any type of expectation for the future need only be conditioned on the current state, rather than a history of any number of states.

$$ P[x_{t+1} | x_{t}] = P[x_{t+1} | x_{t}, x_{t-1}, ..., x_{1}] $$

For the processes we are discussing, $x$ from above can be replaced with $s$ to represent a state. **State transition probability** is thus the probability of being at a new state $s'$ given the current state $s$.
$$P[s' | s] = P_{ss'}$$
read as "probability from s to s'".

Given $n$ states, the probability of obtaining any state given the current state can be summarized as a matrix. We have $n$ rows pertaining to each possible option of the current state, and $n$ columns pertaining to the next state. For example, entry $P_{2, 3}$ represents the probability we move to state 3 given we are at state 2.

## Markov Process

A Markov Process is simply a series of states and transitions defined by a state transition probability matrix $P$. It must maintain the Markov property. An example possible series states (i.e. transitions with non-zero probabilities), is called an episode and finishes upon arrival to a terminal state (zero probability to transition anywhere $P_{Ts} = 0$ for all $s$).  

## Markov Reward Process

Same as above, except now there are rewards associated with each state. Existing or arriving in any state is associated with some reward value. Typically, this framework is used to represent the relative value of a given episode of states and transitions. $R_s$ is usually used to define the expected reward given the current state $s$. That is:

$$R_s = E[R_{t+1} | S_t = s]$$

Read this as the immediate reward associated with state $s$.

It is also useful to have a sense of discounting future rewards when we consider multi-period situations. This is analogous to the time-value discounting of money. This is useful in MPs to account for uncertainty in the future, to avoid cycles in certain processes (as there is now decay), and to accurately represent the present value of certain types of real world rewards (such as money). A factor $\gamma \in [0, 1]$ is used to discount future rewards. It's application can be seen clearly in calculating total return over a time period:
$$G_t = R_{t+1} + \gamma \cdot R_{t+2} + ... = \sum_{k=0}^{\inf} \gamma^{k} \cdot R_{t + k + 1} $$

Read this as the total reward over all future time periods from now, point $t$. Future rewards are discounted by the discount factor $\gamma$ described above. This function describes a particular episode or set of future positions. It is useful to instead use this function to describe the value of a state $s$. Given a state $s$, we would like to know the expected return of existing at this state. Thus, we can take the expectation across all return values associated with this $s$ to describe the value of the state. The value function is:
$$v(s) = E[G_t | S_t = s]$$

Observing G from above, it is clear this function depends on the value of $\gamma$. For example, if $\gamma = 0$, then all future rewards will be discounted to nothing. Thus, the value function for any given state is exactly equal to its immediate reward. On the other hand, $\gamma = 1$ values future rewards as highly as current rewards and the value function may be more complex to calculate.

Upon further evaluation of the value function, it can be decomposed into two separate parts. The immediate reward, and all future rewards. But, notice that the future rewards can be expressed as the discounted value function of the next state $S_{t+1}$. Mathematically:

$$
\begin{align}
v(s) & = E[G_t | S_t = s] \\
& = E[R_{t+1} + \gamma \cdot R_{t+2} + ... | S_t = s] \\
& = E[R_{t+1} + \gamma \cdot G_{t+1} | S_t = s] \\
& = E[R_{t+1} + \gamma \cdot v(S_{t+1}) | S_t = s] \\
\end{align}
$$

Fortunately, we can use the state transition probability matrix to compute the expectation of the value function across possible future states. Notice the first part of the equation $E[R_{t+1} | S_t = s]$ is simply our earlier description of the immediate reward. We can now rewrite our value function as:

$$
\begin{align}
v(s) & = E[R_{t+1} + \gamma \cdot v(S_{t+1}) | S_t = s] \\
& = E[R_{t+1} | S_t = s] + E[\gamma \cdot v(S_{t+1}) | S_t = s] \\
& = R_s + \gamma \sum_{s' \in S} P_{ss'} \cdot v(s')
\end{align}
$$

This is the ever important **Bellman Equation** for MRPs.

The above line decomposes the value function into the two components. First, there is the immediate reward for the state we are evaluating at $s$. Next, we discount the value obtained at the next state, and compute this value by summing across all possible next states in accordance with their probability of occurring. Note the recursive nature of this formula. The formula can also be expressed recursively as 

$$v = R + \gamma P v$$

where v and R are column vectors with one entry per state. The state probability matrix P multiplies each row by the column vector v to sum across possible next states value functions. This makes the Bellman Equation a linear problem, which can be solved directly. 

$$
\begin{align}
v & = R + \gamma P v \\
v - \gamma P v & = R \\ 
(I - \gamma P) v & = R \\
v & = (I - \gamma P)^{-1} + R
\end{align}
$$

But this has complexity $O(n^3)$ where n is the number of states. Iterative methods may be used instead.

## Markov Decision Process

MDP is very similar to MRP, however it adds the concept of **actions**. Actions are elements that take us from one state to another. Incorporating the concept of action in our model allows us to better describe how we move, or may move, from one state to another. Naturally, our representations of state transitions should now incorporate actions. 

$$ P_{ss'}^{a} = E[S_{t+1} = s' | A_t = a, S_t = s] $$
$$ R_s^a = E[R_{t+1} | A_t = a, S_t = s] $$

The key here is that the probability we reach state $s'$ is not only dependent on the state we are in, but also the action that we take from that state. Additionally, the immediate reward is dependent on the action we take. We should then have a representation to describe what actions are possible, or how actions are likely to be taken, given a state, in order for this definition to also make sense. A policy $\pi$ is used to describe the probability distribution of actions given a state.

$$\pi(a|s) = P[A_t = a | S_t = s] $$

Note that this is Markov (only depends on the current state) and also stationary (does not depend on the value of $t$). Policies define the behavior of an agent in a MP. That is, given a MP, the existence of a policy defines how an agent will move from state to state and thus operate in the MP. Note how different policies will have different value functions, as the likelihood of being in different states changes with a change in policy.

Notice also how we can move backwards from this level of abstraction by defining our earlier components of a MP or a MRP for a given/specific policy. That is, once we choose a policy $\pi$, all of our earlier components are defined as they were before. Take for example just a sequence of states and probabilities, i.e. a MP. This is defined by $S$ and $P^\pi$ once $\pi$ is defined. Similarly, for a Markov reward process, we can reproduce our earlier P and R by defining a policy and then summing across actions. Similarly, the value function is simply defined by policy now as well:

$$ P_{ss'}^\pi = \sum_{a \in A} \pi(a|s) \cdot P_{ss'}^a $$
$$ R_s^\pi = \sum_{a \in A} \pi(a|s) \cdot R_{s}^a $$
$$ v_\pi (s) = E[G_t | S_t = s] $$

We can also define the value function to not only account for the current state but also the current action for a bit more granularity. This is called the action-value function and is useful for certain computational features of an MRP. It is defined as:

$$q_\pi (a, s) = E[G_t | S_t = s, A_t = a] $$

Note how the value function can be obtained by summing across all actions in the action-value function and incorporating the probability of taking each action

$$v_\pi(s) = \sum_{a \in A} q_\pi(a, s) \cdot \pi(a|s)$$

The **Bellman Expectation Equation** is the MDP version of the Bellman equation and can be written by summing across the possible actions from a state $s$:

$$v_\pi(s) = \sum_{a \in A} \pi(a | s) \cdot [R_s^a + \gamma \sum_{s' \in S} P_{ss'}^a \cdot v_\pi(s')]$$

A corrolary can also be written for the action-value function:

$$ q_\pi(a, s) = R_s^a + \gamma \sum_{s' \in S} P_{ss'}^a \cdot v_\pi(s') $$

Notice that the Bellman expectation equation above for the value function uses this definition to sum across all actions from state $s$.



It may also be useful to have this equation written instead only in terms of the action-value function. Right now, the RHS still incorporates the current value function. To remedy this, we will again use our definition of translating between value and action-value function by summing across all actions.

$$ q_\pi(a, s) = R_s^a + \gamma \sum_{s' \in S} P_{ss'}^a \sum_{a' \in A} \pi(a'|s') \cdot q_\pi(a', s') $$

This is the same definition as above, except the future value portion is written as a sum of future action-value functions rather than simply the value function.

The Bellman Expectation Equation can be reduced to the Bellman Equation by selecting a policy $\pi$. Remember, this simply redefines our problem now as an MDP. This means that it can once again be linearly solved:

$$ v_\pi = (I - \gamma P^\pi)^{-1} + R^\pi $$

An important definition is the optimal state-value function - this is the maximum value function across all possible policies. Remember, a value function in an MRP is defined by the policy pertaining to it. If we look across all possible policies, we should find a value function that is the maximum for this given state. The same reasoning applies for the action-value function:

$$ v_* (s) = \max_{\pi} v_\pi (s) $$
$$ q_* (a, s) = \max_{\pi} q_\pi (s, a) $$

We consider an MDP solved when we find the policy in the MRP that maximizes the value function, and therefore when we have found the optimal state-value function for each state. There is a theorem that states there is an optimal policy that is better than or equal to all other policies - that is, $v_{\pi^*} \geq v_{\pi}$ for all pi. The theorem does need this policy to be unique, but any optimal policy must obtain the optimal value function for both the value function and the action-value function.

An optimal policy can be found by maximizing over the action-value functions. That is, given state $s$, our policy is to pick action $a$ if it maximizes the action-value function. That is 

$$\pi_*(a | s) = 1 ~~\mathrm{if}~~ a = \mathrm{argmax}~ q_* (a, s)$$
$$ = 0 ~ \mathrm{otherwise} $$

We only select action $a$ if it is the value maximizing action for that state $s$.

Using this concept of an optimal policy, we can extend the **Bellman Expectation Equation** to the Bellman Optimality Equation. The following assert how the optimal value for any given state or state-action pair can be found by following the optimal policy for all future states.

$$
\begin{align}
v_*(s) = \max_a R_s^a + \gamma \sum_{s' \in S} P_{ss'}^a \cdot v_*(s') \\
v_*(s) = \max_a R_s^a + \gamma \sum_{s' \in S} P_{ss'}^a \cdot \max_{a'} q_*(s', a') 
\end{align}
$$

Notice how this function no longer sums across all possible actions, but instead takes only the maximizing action at that state. This checks out with our earlier definition of the optimal policy being the one that takes the maximizing action for each state. Solving this equation is non-linear and requires iterative methods as described in future lectures/notes.

## Code/Data Structures

In [25]:
#this code block is used to define types used later 

from typing import TypeVar, Generic
from typing import Mapping, Set, List, Tuple
import numpy as np

S = TypeVar('S')

# Markov Process

The Markov Process consists of a set of states and state transition probabilities to dictate how one might move from one state to another. In constructing this class, I'd first like to make sure that states are represented generically. The only other data-type is a probability, which can surely be represented consitently as a float. It makes the most sense to me to represent this data-structure as a mapping from state1 to state2 to float, representing the probability of moving from state1 to state2. The set of states is thus easily maintained by the unique keys of the outer dictionary unioned with the unique keys of the inner dictionary. I've also provided a function and a member variable to represent the state probability transition matrix.

NB: My type definitions here are very similar to the sample code from the course github. I took a look at the code first to better understand generic typing / type definitions in python, however attempted to limit looking at any functionality of the code in order to perform the exercise as independently as possible.

In [62]:
class MP(Generic[S]):
    
    # stp = state transition probability
    def __init__(self, stp: Mapping[S, Mapping[S, float]] = None) -> None:
        self.stp_ = stp
        # a list to preserve order for transition matrix
        self.states_ = stp.keys()
        self.mat_ = self.tr_matrix()
    
    def get_transitions(self, state: S) -> Mapping[S, float]:
        return self.stp_[state]
    
    def get_inbound(self, state: S) -> Set[S]:
        
        states = []
        for in_state in self.stp_.keys():
            if state in self.stp_[in_state].keys():
                states.append(in_state)
                
        return states
    
    def update_transitions(self, state: S, transitions: Mapping[S, float]) -> None:
        
        total = 0
        for v in transitions.values():
            assert v >= 0, "Probabilities must be non-negative"
            total += v
        assert total == 1, "Probabilities must sum to 1"
        self.stp_[state] = transitions
        
    def add_state(self, state: S, out: Mapping[S, float], inbound: List[Tuple[S, Mapping[S, float]]]) -> None:
        
        self.update_transitions(state, out)
        for s, tr in inbound:
            self.update_transitions(s, tr)
        self.states_.add(state)
        
    def remove_state(self, state: S, inbound: List[Tuple[S, Mapping[S, float]]]) -> None:
        
        inbound_states = set(self.get_inbound(state))
        arg_states = set()
        for s, tr in inbound:
            arg_states.add(s)
        assert arg_states == inbound_states, "Incorrect replacement set of inbound states"
        
        for s, tr in inbound:
            self.update_transitions(s, tr)
            
        self.states_.remove(state)
        
    def tr_matrix(self) -> np.ndarray:
        
        dim = len(self.states_)
        mat = np.zeros((dim, dim))
        for i, state1 in enumerate(self.states_):
            for j, state2 in enumerate(self.states_):
                try:
                    mat[i][j] = self.stp_[state1][state2]
                except:
                    mat[i][j] = 0
        return mat
    
    def sink_states(self) -> Set[s]:
        states = set()
        for k in self.stp_.keys():
            if len(self.stp_[k]) == 1:
                states.add(k)
        return states

### Testing

In [63]:
transitions = {
        1: {1: 0.1, 2: 0.6, 3: 0.1, 4: 0.2},
        2: {1: 0.25, 2: 0.22, 3: 0.24, 4: 0.29},
        3: {1: 0.7, 2: 0.3},
        4: {1: 0.3, 2: 0.5, 3: 0.2},
        5: {5: 1.0}
    }
mp_obj = MP(transitions)
print(mp_obj.stp_)
print(mp_obj.states_)
print(mp_obj.sink_states())
# stationary = mp_obj.get_stationary_distribution()
# print(stationary)

{1: {1: 0.1, 2: 0.6, 3: 0.1, 4: 0.2}, 2: {1: 0.25, 2: 0.22, 3: 0.24, 4: 0.29}, 3: {1: 0.7, 2: 0.3}, 4: {1: 0.3, 2: 0.5, 3: 0.2}, 5: {5: 1.0}}
dict_keys([1, 2, 3, 4, 5])
[5]


## Markov Reward Process

The Markov Reward Process adds the concept of rewards to the prior Markov Process. The first goal of the code is to utilize the existing infrastructure for the MP. That is, it should be easy to translate from MP to MRP and vice versa. In order to do this, I'm going to use the same structure as before, however append a reward to each state in the dictionary. I am going to use the MP class to define most of the data, and then simply add another mapping to define the immediate rewards for each state. Note that this is the $R_s$ representation. I assume input data is stored as a mapping from state to a tuple of a mapping and a value (possible transitions and immediate reward).

Note the functions for only obtaining the non-terminal states. This is because the reward, and all future reward, of arriving in a non-terminal state is 0. Thus, it is unnecessary to represent these states in the matrix representation that is used to solve for the value function. Even if a state transitions to a terminal state with non-zero probability, the value obtained from this possible transition is always zero and need not be included. Terminal states are represented as a set as order does not matter.

In [80]:
class MRP(MP):
    
    def __init__(self, data: Mapping[S, Tuple[Mapping[S, float], float]], gamma: float) -> None:
        
        # dictionaries to store data
        stp = {}
        rew = {}
        for state in data.keys():
            stp[state] = data[state][0]
            rew[state] = data[state][1]
        super().__init__(stp)
        self.rewards_ = rew
        self.gamma_ = gamma
        # a list to preserve order for transition matrix
        self.nt_states_ = self.get_nt_states()
        # same order as nt states
        self.rvec_ = self.rewards_vec()
 
    def change_reward(self, state: S, r: float) -> None:
        self.rewards_[state] = r
        
    def remove_state(self, state: S, inbound: List[Tuple[S, Mapping[S, float]]]) -> None:
        super().remove_state(state, inbound)
        self.rewards_.pop(state, None)
        
    def add_state(self, state: S, out: Mapping[S, float], inbound: List[Tuple[S, Mapping[S, float]]], r: float) -> None:
        super().add_state(state, out, inbound)
        self.rewards_[state] = r
        
    def rewards_vec(self) -> np.array:
        r = [0] * len(self.nt_states_)
        for i, state in enumerate(self.nt_states_):
            r[i] = self.rewards_[state]
        return np.asarray(r)
    
    def get_terminal_states(self) -> Set[S]:
        term_states = super().sink_states()
        return {s for s in term_states if self.rewards_[s] == 0}
    
    def get_nt_states(self) -> List[S]:
        t_states = self.get_terminal_states()
        return [s for s in self.states_ if s not in t_states]

### Testing

In [83]:
data = {
        1: ({1: 0.6, 2: 0.3, 3: 0.1}, 7.0),
        2: ({1: 0.1, 2: 0.2, 3: 0.7}, 10.0),
        3: ({3: 1.0}, 0.0)
    }
mrp_obj = MRP(data, 1.0)
print(mrp_obj.mat_)
print()
print(mrp_obj.rvec_)
print()
terminal = mrp_obj.get_terminal_states()
print(terminal)
#value_func_vec = mrp_obj.get_value_func_vec()
#print(value_func_vec)

[[0.6 0.3 0.1]
 [0.1 0.2 0.7]
 [0.  0.  1. ]]

[ 7. 10.]

{3}


## Markov Decision Process

MDP